## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-05-21-00-21 +0000,nypost,MTA adds more bus cams to catch driver scofflaws,https://nypost.com/2026/02/05/us-news/mta-adds...
1,2026-02-05-20-55-20 +0000,nyt,What to Know About the Disappearance of Savann...,https://www.nytimes.com/2026/02/03/us/savannah...
2,2026-02-05-20-54-43 +0000,nypost,Vance tells Nazi-loving troll Nick Fuentes to ...,https://nypost.com/2026/02/05/us-news/jd-vance...
3,2026-02-05-20-52-00 +0000,wsj,Bank of Canada Gov. Macklem Warns of Misdiagno...,https://www.wsj.com/articles/bank-of-canada-go...
4,2026-02-05-20-51-43 +0000,nypost,Derrick Callella charged and ID’d as Nancy Gut...,https://nypost.com/2026/02/05/us-news/derrick-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2474/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,30
458,epstein,24
60,new,18
238,up,13
50,olympics,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2026-02-05-20-27-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,76
291,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...,69
274,2026-02-05-03-27-44 +0000,bbc,Trump says America should move on from Epstein...,https://www.bbc.com/news/articles/cy4gj71z0m0o...,66
97,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...,63
313,2026-02-04-23-39-03 +0000,cbc,How the Epstein files unravelled a senior Brit...,https://www.cbc.ca/news/world/epstein-files-ma...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,76,2026-02-05-20-27-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...
291,67,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...
224,62,2026-02-05-11-00-00 +0000,nypost,Pro-ICE billboards calling out ‘liberal hypocr...,https://nypost.com/2026/02/05/us-news/pro-ice-...
118,52,2026-02-05-17-15-17 +0000,nypost,"Stocks plunge on AI spending fears, unemployme...",https://nypost.com/2026/02/05/business/stocks-...
310,48,2026-02-04-23-48-00 +0000,wsj,Federal Reserve governor Lisa Cook sees a grea...,https://www.wsj.com/economy/central-banking/fe...
292,43,2026-02-05-00-53-35 +0000,nypost,Nike under fed scrutiny for DEI initiatives th...,https://nypost.com/2026/02/04/business/nike-un...
279,38,2026-02-05-03-00-43 +0000,nypost,Gavin Newsom bragged about High-Speed Rail ‘st...,https://nypost.com/2026/02/04/us-news/gavin-ne...
67,36,2026-02-05-19-11-58 +0000,nypost,Sheriff refuses to rule out Savannah Guthrie’s...,https://nypost.com/2026/02/05/us-news/sheriff-...
154,35,2026-02-05-15-27-00 +0000,wsj,The Bank of England and ECB kept interest rate...,https://www.wsj.com/economy/central-banking/ba...
96,34,2026-02-05-17-51-02 +0000,nyt,Los Angeles Civic Leader Austin Beutner Drops ...,https://www.nytimes.com/2026/02/05/us/los-ange...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
